## MVP Engenharia de Dados - Notebook 4: Análise (SQL)

**Objetivo:** Responder às perguntas de negócio consultando o modelo de dados final (`default.gold_vendas_flat_model`).

**Período Analisado:** 01/01/2025 a 13/11/2025.

**As 7 Perguntas de Negócio:**

**1. Quais são os 10 produtos mais vendidos (em quantidade)?**
> **Nota de Análise:** O item "VALOR DO SERVIÇO" foi excluído das consultas de produtos por se tratar de uma taxa operacional.

**2. Dentre os diversos tipos de opções, quais os Grupos que vendem mais?**
> **Nota de Análise:** Agrupei os produtos de acordo com a categorização dos alimentos e bebidas.

**2.  Quais são os 10 produtos menos vendidos (em quantidade)?**

**3.  Qual o ranking de faturamento por PDV (bar)?**

**4.  Qual é o faturamento por hora do dia (para identificar picos)?**

**5.  Qual é o valor médio por nota fiscal (Ticket Médio)?**

**6.  Quais são os 10 sócios (ID_SOCIO) que mais consumiram (em valor)?**

**7.  Qual é a divisão do faturamento total e do volume de notas entre os tipos de consumo (`A FATURAR` vs. `PAGO NA HORA`)?**

In [0]:
%sql
-- Verificação do período dos dados
SELECT 
    MIN(DATA_HORA) as Data_Inicial,
    MAX(DATA_HORA) as Data_Final,
    DATEDIFF(MAX(DATA_HORA), MIN(DATA_HORA)) as Total_Dias_Operacao
FROM default.gold_vendas_flat_model;

In [0]:
%sql
-- 1. Top 10 Produtos (Volume)
SELECT 
  NOME_PRODUTO,
  NOME_GRUPO,
  SUM(QUANTIDADE) as Total_Vendido
FROM default.gold_vendas_flat_model
WHERE NOME_GRUPO <> 'TAXA DE SERVIÇO'
GROUP BY NOME_PRODUTO, NOME_GRUPO
ORDER BY Total_Vendido DESC
LIMIT 10;

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- 2. Ranking de Categorias (Grupos)
SELECT 
  NOME_GRUPO,
  COUNT(DISTINCT COD_PRODUTO) as Qtd_Produtos_No_Grupo,
  SUM(FATURAMENTO_LIQUIDO) as Faturamento_Total
FROM default.gold_vendas_flat_model
WHERE NOME_GRUPO <> 'TAXA DE SERVIÇO' -- <--- Ajustado
GROUP BY NOME_GRUPO
ORDER BY Faturamento_Total DESC;

In [0]:
%sql
-- 2. Mix de Vendas Detalhado (Categorização Otimizada)
SELECT 
  CASE 
    -- === 1. BEBIDAS (MACRO) ===
    WHEN NOME_GRUPO LIKE '1.%' THEN 'BEBIDAS (S/ ÁLCOOL)'
    WHEN NOME_GRUPO LIKE '2.%' THEN 'BEBIDAS (ALCOÓLICAS)'
    
    -- === 2. COMIDAS (DETALHADO POR CÓDIGO) ===
    WHEN NOME_GRUPO LIKE '3.01%' THEN 'AVES'
    WHEN NOME_GRUPO LIKE '3.02%' THEN 'CARNES'
    WHEN NOME_GRUPO LIKE '3.03%' THEN 'PEIXES E FRUTOS DO MAR'
    WHEN NOME_GRUPO LIKE '3.04%' THEN 'MASSAS E RISOTOS'
    WHEN NOME_GRUPO LIKE '3.05%' THEN 'OVOS E OMELETES'
    WHEN NOME_GRUPO LIKE '3.06%' THEN 'ENTRADAS E SALADAS'
    WHEN NOME_GRUPO LIKE '3.07%' THEN 'PETISCOS E PORÇÕES'
    WHEN NOME_GRUPO LIKE '3.08%' THEN 'GUARNIÇÕES'
    WHEN NOME_GRUPO LIKE '3.09%' THEN 'JAPONÊS'
    WHEN NOME_GRUPO LIKE '3.10%' THEN 'SOPAS'
    WHEN NOME_GRUPO LIKE '3.11%' THEN 'MOLHOS'
    WHEN NOME_GRUPO LIKE '3.12%' THEN 'INFANTIL'
    WHEN NOME_GRUPO LIKE '3.13%' THEN 'LANCHES E SANDUÍCHES'
    WHEN NOME_GRUPO LIKE '3.14%' THEN 'CAFÉ DA MANHÃ'
    WHEN NOME_GRUPO LIKE '3.15%' THEN 'BUFFET E EVENTOS'
    
    -- === 3. SOBREMESAS (AGRUPADAS) ===
    WHEN NOME_GRUPO LIKE '3.16%' OR NOME_GRUPO LIKE '3.17%' THEN 'SOBREMESAS'
    WHEN NOME_GRUPO LIKE '3.18%' OR NOME_GRUPO LIKE '3.19%' THEN 'SORVETES'
    
    -- === 4. OUTROS TIPOS ===
    WHEN NOME_GRUPO LIKE '3.20%' THEN 'ENCOMENDAS'
    WHEN NOME_GRUPO LIKE '3.21%' THEN 'SUGESTÕES DO CHEF'
    WHEN NOME_GRUPO LIKE '4.%' THEN 'BOMBONIERE'
    WHEN NOME_GRUPO LIKE '5.%' THEN 'TABACARIA'
    
    -- === 5. TAXAS ===
    WHEN NOME_GRUPO LIKE 'Taxa%' OR NOME_GRUPO LIKE 'TAXA%' THEN 'TAXA DE SERVIÇO'
    
    ELSE 'OUTROS'
  END AS Grupos,
  
  SUM(FATURAMENTO_LIQUIDO) as Faturamento_Total,
  ROUND(SUM(FATURAMENTO_LIQUIDO) * 100 / SUM(SUM(FATURAMENTO_LIQUIDO)) OVER (), 1) as Percentual_Faturamento
  
FROM default.gold_vendas_flat_model
WHERE NOME_GRUPO <> 'TAXA DE SERVIÇO'
GROUP BY Grupos
ORDER BY Faturamento_Total DESC;

Databricks visualization. Run in Databricks to view.